In [1]:
from stockfish import Stockfish
import numpy as np

In [55]:
strong_stockfish = Stockfish(
    path='/home/tmek1244/InternalProjects/stockfish_15_linux_x64_avx2/stockfish_15_x64_avx2',
    depth=18,
    parameters={
        "Threads": 6,
        "Hash": 1024*4
    }
)
weak_stockfish = Stockfish(
    path='/home/tmek1244/InternalProjects/stockfish_15_linux_x64_avx2/stockfish_15_x64_avx2',
    depth=3,
    parameters={
        "Threads": 1,
        "Hash": 16
    }
)

In [56]:
import json
import random

starting_position = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"


def convert_move(move):
    from_position = ord(move[0]) - 97 + 8*(int(move[1])-1)
    to_position = ord(move[2]) - 97 + 8*(int(move[3])-1)

    return from_position * 64 + to_position

def convert_evaluation(evaluation):
    return evaluation["Centipawn"] if evaluation["Centipawn"] is not None else 1000*(evaluation["Mate"]//abs(evaluation["Mate"]))

current_moves = [starting_position]

already_done = set()
ITERATION = 100
MOVES_PER_POSITION = 10
MOVES_IN_OUTPUT = 5

for i in range(ITERATION):
    all_positions = set(current_moves)
    current_moves = []
    data = []
    print(f"MOVE {i}, total postions: {len(all_positions)} ")
    for i, position in enumerate(all_positions):
        weak_stockfish.set_fen_position(position)
        pool = weak_stockfish.get_top_moves(MOVES_PER_POSITION)
        print(i, end=' ')
        for move in pool:
            strong_stockfish.set_fen_position(position)
            strong_stockfish.make_moves_from_current_position([move['Move']])
            data_position = strong_stockfish.get_fen_position()

            if data_position in already_done:
                continue
            
            already_done.add(data_position)
            evaluation = strong_stockfish.get_evaluation()
            data_value = evaluation["value"] if evaluation["type"] == "cp" else 1000*(evaluation["value"]//abs(evaluation["value"]))
            top_moves = strong_stockfish.get_top_moves(MOVES_IN_OUTPUT)
            top_moves_converted = [(convert_move(top_move['Move']), convert_evaluation(top_move)) for top_move in top_moves]

            current_moves.append(data_position)
            data.append([data_position, data_value, top_moves_converted])
    print("\nCopying...")
    with open('../../chess_data/fen_eval_top_5_100x10_unique.json', 'a') as f:
        for move in data:
            f.write(str(move)+'\n')
        
    current_moves = random.sample(current_moves, min(100, len(current_moves)))

MOVE 0, total postions: 1 
0 
Copying...
MOVE 1, total postions: 10 
0 1 2 3 4 5 6 7 8 9 
Copying...
MOVE 2, total postions: 100 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 
Copying...
MOVE 3, total postions: 100 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 
Copying...
MOVE 4, total postions: 100 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72

KeyboardInterrupt: 